
# 🌼 RAG기법의 이해와 적용 - 1차시(24.11.28)

---

In [1]:
pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.19
    Uninstalling langchain-core-0.3.19:
      Successfully uninstalled langchain-core-0.3.19
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.7
    Uninstalling langchain-0.3.7:
      Successfully uninstalled langchain-0.3.7


In [2]:
from langchain_community.document_loaders import RecursiveUrlLoader
DOCS_PAGE='https://blog.vllm.ai/2023/06/20/vllm.html'

In [3]:
loader = RecursiveUrlLoader(DOCS_PAGE)

In [4]:
docs = loader.load()

In [5]:
docs

[Document(metadata={'source': 'https://blog.vllm.ai/2023/06/20/vllm.html', 'content_type': 'text/html; charset=utf-8', 'title': 'vLLM: Easy, Fast, and Cheap LLM Serving with PagedAttention | vLLM Blog', 'description': 'GitHub | Documentation | Paper', 'language': 'en'}, page_content='<!DOCTYPE html>\n<html lang="en"><head>\n  <meta charset="utf-8">\n  <meta http-equiv="X-UA-Compatible" content="IE=edge">\n  <meta name="viewport" content="width=device-width, initial-scale=1"><!-- Begin Jekyll SEO tag v2.8.0 -->\n<title>vLLM: Easy, Fast, and Cheap LLM Serving with PagedAttention | vLLM Blog</title>\n<meta name="generator" content="Jekyll v4.3.3" />\n<meta property="og:title" content="vLLM: Easy, Fast, and Cheap LLM Serving with PagedAttention" />\n<meta name="author" content="Woosuk Kwon*, Zhuohan Li*, Siyuan Zhuang, Ying Sheng, Lianmin Zheng, Cody Yu, Joey Gonzalez, Hao Zhang, and Ion Stoica (* Equal Contribution)" />\n<meta property="og:locale" content="en_US" />\n<meta name="descripti

In [6]:
num_documents = len(docs)
print(f'document 개수: {num_documents}')

document 개수: 1


In [7]:
import torch
from sentence_transformers import SentenceTransformer
N_GPU = torch.cuda.device_count()

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [9]:
model_name = "BAAI/bge-large-en-v1.5"

In [10]:
encoder = SentenceTransformer(model_name, device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [11]:
embedding_dim = encoder.get_sentence_embedding_dimension()
max_seq_length = encoder.get_max_seq_length()

In [12]:
print(f'문장 임베딩 시 생성되는 벡터의 차원 크기: {embedding_dim}')
print(f'모델이 처리할 수 있는 입력 문장의 최대 토큰 길이: {max_seq_length}')

문장 임베딩 시 생성되는 벡터의 차원 크기: 1024
모델이 처리할 수 있는 입력 문장의 최대 토큰 길이: 512


In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import numpy as np

chunk_size = 512  # max_seq_length와 동일
chunk_overlap = np.round(chunk_size*0.1, 0)  # 문맥의 유지가 필요한 경우 chunk 사이에 10~20% 정도 겹치는 부분을 생성
print(f'chunk_size: {chunk_size}, chunk_overlap: {chunk_overlap}')

chunk_size: 512, chunk_overlap: 51.0


In [14]:
# chunk_overlap 예시
long_text = '안녕하세요 아주 긴 텍스트를 나눠보겠습니다. 문장 단위로 어떻게 나누어야 할까요?'
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 20,
    chunk_overlap = 5
)
text_chunks = text_splitter.split_text(long_text)
for chunk in text_chunks:
    print(chunk)

안녕하세요 아주 긴 텍스트를
텍스트를 나눠보겠습니다. 문장
문장 단위로 어떻게 나누어야
나누어야 할까요?


In [15]:
child_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size,
    chunk_overlap = chunk_overlap
)

In [16]:
chunks = child_splitter.split_documents(docs)
print(f'{len(docs)}개의 docs가 {len(chunks)}개의 문자로 나뉘었다.')

1개의 docs가 52개의 문자로 나뉘었다.


In [17]:
string_list = []
for doc in chunks:
    if hasattr(doc, 'page_content'):
    # hasattr(객체, 속성명) -> 객체에 속성명이 있는지
        string_list.append(doc.page_content)

In [18]:
embeddings = torch.tensor(encoder.encode(string_list))
embeddings

tensor([[-0.0234,  0.0221, -0.0177,  ..., -0.0006, -0.0043,  0.0399],
        [ 0.0171,  0.0109, -0.0180,  ..., -0.0039, -0.0265,  0.0146],
        [ 0.0079,  0.0002,  0.0203,  ..., -0.0080, -0.0263,  0.0426],
        ...,
        [ 0.0074,  0.0103,  0.0246,  ...,  0.0150, -0.0150,  0.0111],
        [-0.0002,  0.0002,  0.0041,  ...,  0.0242, -0.0120, -0.0135],
        [-0.0334, -0.0142,  0.0187,  ..., -0.0048, -0.0167,  0.0039]])

In [19]:
embeddings = np.array(embeddings / np.linalg.norm(embeddings))
# 벡터 정규화: 상대적인 값을 유지하며 크기를 조정하여 유사도, 거리 등을 비교하기 위해
# linalg.norm: 벡터의 크기를 계산
embeddings

array([[-3.2433148e-03,  3.0656536e-03, -2.4589787e-03, ...,
        -7.9131096e-05, -5.9362175e-04,  5.5376114e-03],
       [ 2.3663254e-03,  1.5079604e-03, -2.4914653e-03, ...,
        -5.3539872e-04, -3.6686282e-03,  2.0307114e-03],
       [ 1.0944124e-03,  3.0741558e-05,  2.8103061e-03, ...,
        -1.1119934e-03, -3.6409216e-03,  5.9099812e-03],
       ...,
       [ 1.0273707e-03,  1.4269428e-03,  3.4093512e-03, ...,
         2.0777669e-03, -2.0858389e-03,  1.5358157e-03],
       [-3.2893062e-05,  2.6786507e-05,  5.6232640e-04, ...,
         3.3581476e-03, -1.6645045e-03, -1.8742131e-03],
       [-4.6384879e-03, -1.9680921e-03,  2.5938142e-03, ...,
        -6.6451193e-04, -2.3227520e-03,  5.3908327e-04]], dtype=float32)

In [20]:
converted_values = list(map(np.float32, embeddings))
# float32: CPU, GPU에서 빠른 속도를 낸다
converted_values

[array([-3.2433148e-03,  3.0656536e-03, -2.4589787e-03, ...,
        -7.9131096e-05, -5.9362175e-04,  5.5376114e-03], dtype=float32),
 array([ 0.00236633,  0.00150796, -0.00249147, ..., -0.0005354 ,
        -0.00366863,  0.00203071], dtype=float32),
 array([ 1.0944124e-03,  3.0741558e-05,  2.8103061e-03, ...,
        -1.1119934e-03, -3.6409216e-03,  5.9099812e-03], dtype=float32),
 array([ 1.4564579e-03,  4.7587943e-03, -1.1864128e-03, ...,
         4.6959500e-05, -3.8631826e-03,  4.2156610e-03], dtype=float32),
 array([-0.00234334,  0.00301518, -0.00062454, ..., -0.00045515,
        -0.00390585,  0.00814049], dtype=float32),
 array([ 0.00089627,  0.00172723, -0.00284757, ..., -0.00452556,
        -0.00457108,  0.00146858], dtype=float32),
 array([ 0.00058679,  0.00072843,  0.00045178, ..., -0.00290517,
        -0.00711539,  0.00377408], dtype=float32),
 array([ 0.00090235, -0.00144551,  0.003178  , ..., -0.00347104,
        -0.00448765, -0.00245557], dtype=float32),
 array([-0.0028666

In [21]:
dict_list = []
for chunk, vector in zip(chunks, converted_values):
    # chunk: 현재 테스트 조각
    # vector: 해당 테스트 조각의 임베딩 벡터
    chunk_dict = {
        'chunk': chunk.page_content,
        'source': chunk.metadata.get('source', ''),
        'vector': vector
    }
    dict_list.append(chunk_dict)

In [22]:
chunk_dict

{'chunk': '<div class="social-links"><ul class="social-media-list"></ul>\n</div>\n\n  </div>\n\n</footer>\n</body>\n\n</html>',
 'source': 'https://blog.vllm.ai/2023/06/20/vllm.html',
 'vector': array([-0.00463849, -0.00196809,  0.00259381, ..., -0.00066451,
        -0.00232275,  0.00053908], dtype=float32)}

```
chunks = [
    Document(page_content = '첫 번째 본문', metadata= {'source': 'doc1.com'}),
    Document(page_content = '두 번째 본문', metadata = {'source': 'doc2.com'})
]

converted_value = [
    [0.1, 0.2, 0.3],
    [0.4, 0.5, 0.6]
]
```

```
{
    'chunk': '첫 번째 본문',
    'source': 'doc1.com',
    'vector': [0.1, 0.2, 0.3]
}
```

In [23]:
pip install pymilvus

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 111.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.68.0
    Uninstalling grpcio-1.68.0:
      Successfully uninstalled grpcio-1.68.0


In [24]:
from pymilvus import MilvusClient
import time

In [25]:
mc = MilvusClient('milvus_test.db')

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: 246a953ea88f41c7a2f235a8776e3db3


In [26]:
collection_name = 'milvus_docs'
# CRUD: Create, Read, Update, Delete

In [27]:
mc.create_collection(collection_name,
                     embedding_dim,
                     consustency_level = 'Eventually',
                     # 데이터의 일관성 수준, 보통은 Eventually 사용 (강한 일관성: 금융, 의료 등 실시간으로 반영해야 하는 데이터 - Strong)
                     auto_id = True,
                     # 각자 알아서 인덱스를 자동으로 부여
                     overwrite = True)
                     # 이미 존재하는 collection_name 이름의 collection이 있다면 덮어써라

DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: milvus_docs
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: milvus_docs


In [28]:
print('데이터 삽입 시작!')
start_time = time.time()

mc.insert(
    collection_name,
    data = dict_list,
    progress_bar = True
)

end_time = time.time()
print(f'삽입된 데이터(벡터)의 개수: {len(dict_list )}', end=' ')
print(f'삽입에 걸린 시간: {round(end_time-start_time, 2)}초')

데이터 삽입 시작!
삽입된 데이터(벡터)의 개수: 52 삽입에 걸린 시간: 0.1초


In [29]:
import torch.nn.functional as F

In [30]:
sample_question = 'What is PageAttention?'

In [31]:
query_embeddings = torch.tensor(encoder.encode([sample_question]))

In [32]:
query_embeddings = F.normalize(query_embeddings, p=2, dim=1)
query_embeddings

tensor([[-0.0546,  0.0105, -0.0124,  ..., -0.0239, -0.0076,  0.0367]])

In [33]:
query_embeddings = list(map(np.float32, query_embeddings))
query_embeddings

[array([-0.05461461,  0.01051912, -0.01240393, ..., -0.02393391,
        -0.00762213,  0.03665992], dtype=float32)]

In [34]:
output_fields = list(dict_list[0].keys())
output_fields.remove('vector')

In [35]:
top_k = 4

In [36]:
result = mc.search(
    collection_name,
    data=query_embeddings,
    output_fields=output_fields,
    limit=top_k,
    consistency_level='Eventually'
)

In [37]:
result[0][0]['entity']['chunk']

'<p align="center">\n<picture>\n<img src="/assets/figures/annimation3.gif" width="100%" />\n</picture>\n<br />\nExample generation process for a request that samples multiple outputs.\n</p>\n\n<p>PageAttention’s memory sharing greatly reduces the memory overhead of complex sampling algorithms, such as parallel sampling and beam search, cutting their memory usage by up to 55%. This can translate into up to 2.2x improvement in throughput. This makes such sampling methods practical in LLM services.</p>'

In [38]:
result[0][2]['entity']['chunk']

'<p>PagedAttention has another key advantage: efficient memory sharing. For example, in <em>parallel sampling</em>, multiple output sequences are generated from the same prompt. In this case, the computation and memory for the prompt can be shared between the output sequences.</p>\n\n<p align="center">\n<picture>\n<img src="/assets/figures/annimation2.gif" width="80%" />\n</picture>\n<br />\nExample of parallel sampling.\n</p>'

In [39]:
result[0][3]['entity']['chunk']

'Easy, Fast, and Cheap LLM Serving with PagedAttention","mainEntityOfPage":{"@type":"WebPage","@id":"/2023/06/20/vllm.html"},"url":"/2023/06/20/vllm.html"}</script>'

In [40]:
# 가장 유사도 높은 하나의 단일 chunk만 추가하는 경우
# contexts = []
# contexts.append(result[0][0]['entity']['chunk'])

In [41]:
# top_k의 chunk를 모두 추가하는 경우
contexts = []
for i in range(top_k):
    contexts.append(result[0][i]['entity']['chunk'])
contexts
# 같은 값이 들어간 것은 내가 데이터를 세 번 넣었기 때문...

['<p align="center">\n<picture>\n<img src="/assets/figures/annimation3.gif" width="100%" />\n</picture>\n<br />\nExample generation process for a request that samples multiple outputs.\n</p>\n\n<p>PageAttention’s memory sharing greatly reduces the memory overhead of complex sampling algorithms, such as parallel sampling and beam search, cutting their memory usage by up to 55%. This can translate into up to 2.2x improvement in throughput. This makes such sampling methods practical in LLM services.</p>',
 '<p>PagedAttention naturally enables memory sharing through its block table. Similar to how processes share physical pages, different sequences in PagedAttention can share the blocks by mapping their logical blocks to the same physical block. To ensure safe sharing, PagedAttention keeps track of the reference counts of the physical blocks and implements the <em>Copy-on-Write</em> mechanism.</p>',
 '<p>PagedAttention has another key advantage: efficient memory sharing. For example, in <e

In [42]:
contexts_combined = ' '.join(contexts)
contexts_combined

'<p align="center">\n<picture>\n<img src="/assets/figures/annimation3.gif" width="100%" />\n</picture>\n<br />\nExample generation process for a request that samples multiple outputs.\n</p>\n\n<p>PageAttention’s memory sharing greatly reduces the memory overhead of complex sampling algorithms, such as parallel sampling and beam search, cutting their memory usage by up to 55%. This can translate into up to 2.2x improvement in throughput. This makes such sampling methods practical in LLM services.</p> <p>PagedAttention naturally enables memory sharing through its block table. Similar to how processes share physical pages, different sequences in PagedAttention can share the blocks by mapping their logical blocks to the same physical block. To ensure safe sharing, PagedAttention keeps track of the reference counts of the physical blocks and implements the <em>Copy-on-Write</em> mechanism.</p> <p>PagedAttention has another key advantage: efficient memory sharing. For example, in <em>paralle

In [43]:
system_prompt = f"""유사도가 높게 나온 Context: {contexts_combined}"""
prompts = [system_prompt]
prompts

['유사도가 높게 나온 Context: <p align="center">\n<picture>\n<img src="/assets/figures/annimation3.gif" width="100%" />\n</picture>\n<br />\nExample generation process for a request that samples multiple outputs.\n</p>\n\n<p>PageAttention’s memory sharing greatly reduces the memory overhead of complex sampling algorithms, such as parallel sampling and beam search, cutting their memory usage by up to 55%. This can translate into up to 2.2x improvement in throughput. This makes such sampling methods practical in LLM services.</p> <p>PagedAttention naturally enables memory sharing through its block table. Similar to how processes share physical pages, different sequences in PagedAttention can share the blocks by mapping their logical blocks to the same physical block. To ensure safe sharing, PagedAttention keeps track of the reference counts of the physical blocks and implements the <em>Copy-on-Write</em> mechanism.</p> <p>PagedAttention has another key advantage: efficient memory sharing. For ex

In [44]:
from huggingface_hub import login
from google.colab import userdata
hf_token = input('HF')
login(token = hf_token)

HFhf_MaJEGJinjOsOXrtXVOHuDZPYSZWYXJfCTR


In [50]:
pip install vllm

In [46]:
pip install triton

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.5 MB/s eta 0:00:00


In [47]:
torch.cuda.empty_cache()
# gpu cache를 지워준다

In [48]:
import vllm, torch
from vllm import LLM, SamplingParams

In [49]:
import torch
model_torun = "meta-llama/Llama-3.2-1B-Instruct"

In [51]:
llm = LLM(
    model = model_torun,
    enforce_eager=True,  # 즉시
    dtype = torch.float16,
    gpu_memory_utilization=0.9,
    max_model_len=1024
)

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

WARNING 11-28 07:39:27 config.py:1865] Casting torch.bfloat16 to torch.float16.
INFO 11-28 07:39:38 config.py:350] This model supports multiple tasks: {'generate', 'embedding'}. Defaulting to 'generate'.
WARNING 11-28 07:39:38 config.py:503] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 11-28 07:39:38 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='meta-llama/Llama-3.2-1B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-1B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=c

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

...

INFO 11-28 07:39:43 selector.py:261] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 11-28 07:39:43 selector.py:144] Using XFormers backend.
INFO 11-28 07:39:45 model_runner.py:1072] Starting to load model meta-llama/Llama-3.2-1B-Instruct...
INFO 11-28 07:39:46 weight_utils.py:243] Using model weights format ['*.safetensors']


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

INFO 11-28 07:40:45 weight_utils.py:288] No model.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 11-28 07:40:49 model_runner.py:1077] Loading model weights took 2.3185 GB
INFO 11-28 07:40:49 worker.py:232] Memory profiling results: total_gpu_memory=14.75GiB initial_memory_usage=3.74GiB peak_torch_memory=4.75GiB memory_usage_post_profile=3.76GiB non_torch_memory=0.18GiB kv_cache_size=8.34GiB gpu_memory_utilization=0.90
INFO 11-28 07:40:50 gpu_executor.py:113] # GPU blocks: 17090, # CPU blocks: 8192
INFO 11-28 07:40:50 gpu_executor.py:117] Maximum concurrency for 1024 tokens per request: 267.03x


In [52]:
from vllm import SamplingParams
sampling_params = SamplingParams(temperature=0.2, top_p=0.95)

In [53]:
outputs = llm.generate(prompts, sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.20it/s, est. speed input: 417.57 toks/s, output: 19.31 toks/s]


In [54]:
for output in outputs:
  prompt = output.prompt
  generated_text = output.outputs[0].text
  print(f'질문: {sample_question}')
  print(f'생성된 모델의 응답: {generated_text}')

질문: What is PageAttention?
생성된 모델의 응답:  <script>var scriptOptions = {isSinglePage: true, renderParams
